In [ ]:
import os

import sqlite3
import pandas as pd

In [ ]:
# Path to the root directory of datasci
dthon_path = os.path.join(os.path.expanduser('~'), 'Projects','datasci')
dthon_db = os.path.join(dthon_path, "melbourne-datathon","data","medi.db")
# Run this first to create database connection
conn = sqlite3.connect(dthon_db)
c = conn.cursor()

In [ ]:
# Create temp table transactions depression with only transactions including and after 2016
# Takes a bout 5 minutes to create
sql = """
CREATE TEMP TABLE transactions_depression
AS
SELECT * FROM transactions t
WHERE Dispense_Week >= '2016-01-01'
AND t.Drug_ID IN (
    SELECT c.MasterProductID
    FROM ChronicIllness_LookUp c
    WHERE c.ChronicIllness = 'Depression'
)
"""

%time c.execute(sql)

In [ ]:
# Creates table based on previous table with only victorian results. takes a second
sql = """
CREATE TABLE transactions_depression_vic
AS
SELECT * FROM transactions_depression t
WHERE t.Patient_ID IN (
    SELECT Patient_ID
    FROM patients p
    WHERE p.postcode IN (
        SELECT postcode FROM postcodes_geo
        WHERE state = 'VIC'
    )
)
"""
%time c.execute(sql)

In [ ]:
# Creates Create indexes on this table
sql = "CREATE INDEX PatientID_VIC_Index ON transactions_depression_vic(Patient_ID);"
c.execute(sql)
sql = "CREATE INDEX Dispense_Week_VIC_Index ON transactions_depression_vic(Dispense_Week);"
c.execute(sql)
sql = "CREATE INDEX DrugID_VIC_Index ON transactions_depression_vic(Drug_ID);"
c.execute(sql)


In [ ]:
# This query gets pairs of transactions within two weeks of each other that may cause adverse affects
# based on the pairs in Adverse_Interaction table.
# Takes 5000 seconds or 83 minutes with one drug pair in adverse affects
sql = """
SELECT t1.Patient_ID,
    t1.Store_ID as store_id1, t1.Prescriber_ID as prescriber_id1, t1.Drug_ID as drug_id1, t1.Dispense_Week as dispense_week1, 
    t2.Store_ID as store_id2, t2.Prescriber_ID as prescriber_id2, t2.Drug_ID as drug_id2, t2.Dispense_Week as dispense_week2
FROM transactions_depression_vic t1
JOIN transactions_depression_vic t2 ON t1.Patient_ID = t2.Patient_ID
AND t2.Dispense_Week BETWEEN date(t1.Dispense_Week, '-14 days') AND date(t1.Dispense_Week, '+14 days')
AND t1.Drug_ID IN (
    SELECT MasterProductID
    FROM Drug_LookUp
    INNER JOIN Adverse_Interaction ai ON ai.DrugA_ID = GenericIngredientName
)
AND t2.Drug_ID IN (
    SELECT MasterProductID
    FROM Drug_LookUp
    INNER JOIN Adverse_Interaction ai ON ai.DrugA_ID IN (
        SELECT GenericIngredientName
        FROM Drug_LookUp
        WHERE MasterProductID = t1.Drug_ID
    )
    WHERE GenericIngredientName = ai.DrugB_ID
)
"""

%time df = pd.read_sql_query(sql, conn)
df.head()